In [1]:
%matplotlib inline
import os, sys, gc
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr 

In [6]:
def around(mat, dp = 3):
    return np.around(mat, dp)

def array_match_fn(array1, array2):
    for row in array1:
        

loadDir = 'preds_scores_9k'
#print(os.listdir(loadDir))
model = 'shake_shake_26_2x64d_SSI_cutout16'

control = np.load('{0}/{1}_False.npz'.format(loadDir, model))
controlTop = np.argmax(control['probs'], axis = 1)
controlLabels = np.argmax(control['labels'], axis = 1)

actual = np.load('{0}/{1}_True.npz'.format(loadDir, model))
actualTop = np.argmax(actual['probs'], axis = 1)
actualLabels = np.argmax(actual['labels'], axis = 1)

humans = np.load('human_bincounts.npy')
humans = humans / np.sum(humans, axis = 1)[:, np.newaxis]
humanTop = np.argmax(humans, axis = 1)

print(list(control.keys()))

print(around(control['labels'][:5]), '\n', around(humans[:5]), '\n',
     around(actual['labels'][:5]))

print(controlLabels[:5], '\n', humanTop[:5], '\n',
     actualLabels[:5])



['labels', 'logits', 'probs']
[[0.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.02  0.9   0.    0.08  0.    0.    0.    0.   ]
 [0.    0.019 0.    0.    0.    0.    0.    0.    0.962 0.019]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.   ]] 
 [[0.    0.02  0.02  0.941 0.    0.02  0.    0.    0.    0.   ]
 [0.    0.    0.02  0.    0.    0.    0.    0.    0.98  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    1.    0.   ]
 [0.745 0.    0.157 0.    0.02  0.    0.02  0.    0.039 0.02 ]
 [0.    0.    0.02  0.    0.    0.    0.98  0.    0.    0.   ]] 
 [[0.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.02  0.9   0.    0.08  0.    0.    0.    0.   ]
 [0.    0.019 0.    0.    0.    0.    0.    0.    0.962 0.019]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.   ]]
[5 3 8 4 6] 
 [3 8

In [7]:
corrDict = {}
corrDict['corCon'] = pearsonr(controlTop, controlLabels)[0]
corrDict['corConH'] = pearsonr(controlTop, humanTop)[0]
corrDict['actCon'] = pearsonr(actualTop, actualLabels)[0]
corrDict['actConH'] = pearsonr(actualTop, humanTop)[0]

print(corrDict)

{'corCon': 0.8913918331742259, 'corConH': 0.010310335875125292, 'actCon': 0.8746086487506933, 'actConH': 0.003601676147718379}
